In [1]:
from math import pi 
from ngsolve import *
from xfem import *
from netgen.occ import *
from ngsolve.webgui import *

importing ngsxfem-2.1.2504


In [2]:
# Important parameters and functions
## choose A1 = A2 = A_gamma = I 
## choose alpha = xi = 1, then B is also an identity matrix
# f = [2*x*(1-x)+2*y*(1-y), 2*x*(1-x)+2*y*(1-y),1/2]
f = [0,0,0]
gD = [0, 1]
gN = 0
a_gamma = IfPos(y-1/4, 2*10**(-3), 1) - IfPos(y-3/4,2*10**(-3)-1,0)
# 1/4<y<3/4时为2*10**（-3），其余为1
d = 0.01
Af = a_gamma*d
alpha = 2*a_gamma/d

In [3]:
# Construct background mesh
# geo = OCCGeometry(unit_square_shape.Scale((0,0,0),1), dim=2)
# mesh = Mesh(geo.GenerateMesh(maxh=0.4,quad_dominated=False))
Omega = Rectangle(2, 1).Face() 
Omega.faces.name = "Omega"
Omega.edges.Min(X).name = "left"
Omega.edges.Min(Y).name = "bottom"
Omega.edges.Max(X).name = "right"
Omega.edges.Max(Y).name = "top"
mesh = Mesh(OCCGeometry(Omega, dim=2).GenerateMesh(maxh=0.05))
# print(mesh.GetBoundaries())
dirichlet_bnd = 'left|right'
neumann_bnd = 'top|bottom'

In [9]:
# Define the levelset function for the interface
levelset = x-1
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
DrawDC(lsetp1,-1,1,mesh,'lsetp1')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [5]:
# Construct the unfitted fem space 
Vhbase = H1(mesh,order=1,dirichlet=dirichlet_bnd,dgjumps=True)
ci = CutInfo(mesh,lsetp1)
Vh = FESpace([Compress(Vhbase,GetDofsOfElements(Vhbase,ci.GetElementsOfType(cdt))) for cdt in [HASNEG, HASPOS,IF]])
# Vh = FESpace([Restrict(Vhbase,GetDofsOfElements(Vhbase,ci.GetElementsOfType(cdt))) for cdt in [HASNEG, HASPOS,IF]])

In [6]:
# Define averages and jumps

u,v = Vh.TnT()
h = specialcf.mesh_size
n = specialcf.normal(2)
n_gamma = Normalize(grad(lsetp1))
ave_u = (u[0] + u[1])/2
ave_v = (v[0] + v[1])/2
jump_u = (grad(u[0]) - grad(u[1]))*n_gamma
jump_v = (grad(v[0]) - grad(v[1]))*n_gamma

In [7]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
haspos = ci.GetElementsOfType(HASPOS)
hasif = ci.GetElementsOfType(IF)

In [8]:
# Construc bilinear form and right hand side 

## integral operators
dx_neg = dCut(levelset=lsetp1, domain_type = NEG, definedonelements=ci.GetElementsOfType(HASNEG))
dx_pos = dCut(levelset=lsetp1, domain_type = POS, definedonelements=ci.GetElementsOfType(HASPOS))
dgamma = dCut(levelset=lsetp1, domain_type = IF, definedonelements=ci.GetElementsOfType(IF))

## Bilinear form
ah = BilinearForm(Vh,symmetric=True)
ah += grad(u[0]) * grad(v[0]) * dx_neg +  grad(u[1]) * grad(v[1]) * dx_pos + Af*grad(u[2]) * grad(v[2]) * dgamma
# ah += alpha*((u[0]-u[2]) * (v[0]-v[2]) + (u[1]-u[2]) * (v[1]-v[2]) )* dgamma
ah += alpha*(ave_u-u[2])*(ave_v-v[2])* dgamma
# ah += alpha*((u[0]-u[2])*v[0]+(u[1]-u[2])*v[1]+(2*u[2]-u[0]-u[1])*v[2])*dgamma
ah += h * jump_u * jump_v * dgamma 
ah. Assemble()

## right hand side
F = LinearForm(Vh)
F += ( f[0] * v[0] * dx_neg + f[1] * v[1] * dx_pos + f[2] * v[2] * dgamma)
F += gN * v[0]*ds(definedon=mesh.Boundaries(neumann_bnd))
F.Assemble()

# print(ah.mat)

In [10]:
gfu = GridFunction(Vh)
freedofs = Vh.FreeDofs()
## deal with Dirichlet boundary
gfu.components[0].Set(gD[0],BND)
gfu.components[1].Set(gD[1],BND)
F.vec.data -= ah.mat * gfu.vec
gfu.vec.data += ah.mat.Inverse(freedofs) * F.vec

In [11]:
mask0 = IfPos(levelset, 0, 1)   # 只保留界面一边
mask1 = IfPos(levelset, 1, 0)   # 只保留界面一边
uh1 = gfu.components[0]*mask0
uh2 = gfu.components[1]*mask1
uh = uh1 + uh2
Draw(uh,mesh)
Draw(gfu.components[2],mesh)
vtk = VTKOutput(ma=mesh,
                coefs=[uh],      # 可以传入多个函数 [gfu1, gfu2,...]
                names=["pm"],
                filename="/mnt/d/ngs_output/pm_edfm",
                subdivision=2)  
vtk.Do()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

'/mnt/d/ngs_output/pm_edfm'

In [12]:
gfu_orig = GridFunction(Vhbase)
gfu_orig.vec[:] = 0  # 默认设置为 0
used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(HASNEG))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] += gfu.components[0].vec[i]

used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(HASPOS))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] += gfu.components[1].vec[i]

used_dofs = GetDofsOfElements(Vhbase,ci.GetElementsOfType(IF))
# 将压缩空间上的向量嵌入到原始空间
dof_indices = [i for i in range(len(used_dofs)) if used_dofs[i]]
for i, dof in enumerate(dof_indices):
    gfu_orig.vec[dof] -= gfu.components[2].vec[i]

Draw(gfu_orig,mesh)


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [13]:
vtk = VTKOutput(ma=mesh,
                coefs=[gfu_orig,gfu.components[0],gfu.components[1],gfu.components[2]],      # 可以传入多个函数 [gfu1, gfu2,...]
                names=["wholedomain","uh1","uh2","uhf"],
                filename="/mnt/d/ngs_output/l2_solution4",
                subdivision=2)  
vtk.Do()

'/mnt/d/ngs_output/l2_solution4'